# Domain Sorting 
## Author: Damien 

#### I realized during the development of the other code that the multifidelity_network function should work with individual point inputs, not a list of point inputs. I am rewriting the code for the evaluation of the multifidelity network to account for this.

In [ ]:
import numpy as np
from anytree import NodeMixin, RenderTree
import math
import matplotlib.pyplot as plt

In [ ]:
class MFDomain(NodeMixin):
    def __init__(self,vertices,parent=None,children=None):
        super(MFDomain,self).__init__()
        self.vertices = np.array(vertices) # two opposite vertices that define the n-dimensional box
        self.parent = parent

        if children: # set children
            self.children = children

        self.root.tree_level_organizer(self)

    def weight(self,x):
        """
        This is the coefficient weight function of the neural domain
        # NOTE: This function only currently works for 1D functions. I need to implement a high dimensional version
        """
        vertices = self.vertices
        mu = (vertices[0] + vertices[1])/2
        sigma = (vertices[1] - vertices[0])/2
        w = 1+ np.cos(math.pi*(x-mu)/sigma)
        w = w**2
        return w

    def apply_mf(self,pts,u_lf):
        val = np.ones(len(pts))
        return val


In [ ]:
class RootUtilities:
    def __init__(self):
        self.levels = [] # list of what classes are on which levels of the domain tree

    def tree_level_organizer(self,new_node):
        """
        This function recognizes which level "new_node" is on and places the class in the appropriate
        level in "self.levels".
        """
        depth = new_node.depth # find the depth of the new node
        if depth >= len(self.levels): # if "new_node" is the first on a new level, add that level to "self.levels"
            self.levels.append([new_node])
        else: # if "new_node" is a member of an existing level, add it to that level of "self.levels"
            self.levels[depth].append(new_node)

class SFDomain(RootUtilities,NodeMixin):
    def __init__(self,vertices,children=None):
        super(SFDomain,self).__init__()
        self.vertices = np.array(vertices) # two opposite vertices that define the n-dimensional box

        if children: # set children
            self.children = children

        self.pts = np.array([]) # the points on the interior of the root domain
        self.global_indices = np.arange(1) # NOTE: This is a hard-coded batch size. This needs to be changed.

        self.tree_level_organizer(self)
    
    def apply_sf(self,pts): # dummy function to test "evaluate_neural_domain_tree"
        val = np.ones(len(pts))
        return val

    def find_interior_points(self,verts,parent_pts): # TESTED
        """
        Creates a mask that communicates the locations of which points in "pts" are in the hyperrectangle
        defined by verts.
        """
        indices = []
        pts = []
        number_of_points = len(parent_pts)
        for i in range(number_of_points):
            condition = ((verts[0] < parent_pts[i]) & (parent_pts[i] < verts[1])).all()
            if condition:
                indices.append(i)
                pts.append(parent_pts[i])
        return np.array(indices), np.array(pts)
    
    def multifidelity_network(self,params,pts):


    # def evaluate_neural_domain_tree(self,pts): # TESTED
    #     """
    #     Evaluate the neural domain tree layer by layer. 
    #     NOTE: The current code only works if the support of the union of the domains at each level is the original domain.
    #     """
    #     self.pts = pts # save points to class
    #     u_pred = self.apply_sf(pts)
    #     u_preds = np.zeros((len(self.levels),len(pts)))
    #     u_preds[0,:] = u_pred # the first prediction of the solution is the single fidelity net
    #     self.global_indices = np.arange(len(pts)) # NOTE: This is inefficient. Find a better way to set this
    #     iter = 1
    #     for level in self.levels[1:]: # iterates through the levels of the domain tree (skipping the first level which contains the root)
    #         u_pred = np.zeros(len(pts))
    #         for mfdomain in level: # iterates through the MFDomains in each level of the tree. NOTE: parallelize this loop
    #             parent = mfdomain.parent        
    #             local_indices, mfdomain.pts = self.find_interior_points(mfdomain.vertices,parent.pts)
    #             mfdomain.global_indices = parent.global_indices[local_indices] # get the indices of the points in the support of mfdomain
    #             output = mfdomain.apply_mf(mfdomain.pts,u_preds[-1][mfdomain.global_indices]) # analyze the local network
    #             u_pred[mfdomain.global_indices] += output # add the output to the prediction of the solution
    #         u_preds[iter,:] = u_pred
    #         iter += 1
    #     return u_preds